In [ ]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()

In [ ]:
!pip install timm
!pip install mediapipe
!pip install SunoAI
!pip install gradio
!pip install torch transformers==4.40.0 accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 28.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cookies=input('cookies:')
expression_model_path='/content/drive/MyDrive/best_model.pt'
age_model_path='/content/drive/MyDrive/swinv2_ages.pt'
gender_model_path='/content/drive/MyDrive/swinv2_gender.pt'

In [ ]:
import mediapipe as mp
from PIL import Image, ImageOps
import numpy as np

mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils
def detect_and_crop_face(image):
    with mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5) as face_detection:
        image_np = np.array(image)
        results = face_detection.process(image_np)
        if results.detections:
            detection = results.detections[0]
            bbox = detection.location_data.relative_bounding_box
            ih, iw, _ = image_np.shape
            xmin = int(bbox.xmin * iw)
            ymin = int(bbox.ymin * ih)
            width = int(bbox.width * iw)
            height = int(bbox.height * ih)
            xmax = xmin + width
            ymax = ymin + height
            face = image.crop((xmin, ymin, xmax, ymax))
            return face
        else:
            return -1




In [ ]:
from torchvision.transforms import Compose, Resize, Normalize, ToTensor
expression_test_transform = Compose([
    Resize((224, 224)),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
age_test_transform=Compose([
    Resize((256,256)),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])
gender_test_transform=Compose([
    Resize((256,256)),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225))
])

In [ ]:
import transformers
import torch

model_id = "MLP-KTLim/llama-3-Korean-Bllossom-8B"

pipeline = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={"torch_dtype": torch.bfloat16},
    device_map="auto",
)

pipeline.model.eval()




/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/703 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/51.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/10.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/301 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(145088, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head)

In [ ]:
import os
import gradio as gr
import torch
from PIL import Image, ImageOps
from torchvision.transforms import Compose, Resize, Normalize, ToTensor

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
expression_model = torch.load(expression_model_path, map_location=device)
age_model = torch.load(age_model_path, map_location=device)
gender_model = torch.load(gender_model_path, map_location=device)
expression_model.eval()
age_model.eval()
gender_model.eval()
expression_model.to(device)
age_model.to(device)
gender_model.to(device)
label = ['화난', '행복한', '혼란스러운', '슬픈']
gender = ['남자', '여자']



In [ ]:
import suno
def llama_run(text):
    PROMPT = '''당신은 유용한 생성형 AI 어시스턴트입니다. 사용자의 질의에 대해 친절하고 정확하게 답변해야 합니다.'''
    instruction = f"'{text}'에게 어울리는 음악 장르와 분위기만 제안해줘."
    messages = [
        {"role": "system", "content": f"{PROMPT}"},
        {"role": "user", "content": f"{instruction}"}
        ]

    prompt = pipeline.tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
    )

    terminators = [
        pipeline.tokenizer.eos_token_id,
        pipeline.tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = pipeline(
        prompt,
        max_new_tokens=350,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.5,
        top_p=0.9
    )

    return outputs[0]["generated_text"][len(prompt):]
def model_run(img):
    if img is None:
        return "이미지를 업로드하세요.", None

    try:
        gr.Info('이미지 분석 중!!!')
        with torch.no_grad():
            img = ImageOps.exif_transpose(img)
            cropped_img = detect_and_crop_face(img)
            if cropped_img == -1:
                return "얼굴을 찾을 수 없습니다.", None

            expression_img = expression_test_transform(cropped_img)
            age_img = age_test_transform(cropped_img)
            gender_img = gender_test_transform(cropped_img)

            expression_img = expression_img.unsqueeze(0).to(device)
            age_img = age_img.unsqueeze(0).to(device)
            gender_img = gender_img.unsqueeze(0).to(device)

            expression_pred = expression_model(expression_img)
            age_pred = age_model(age_img)
            gender_pred = gender_model(gender_img)

            expression_pred = label[expression_pred.argmax(1).detach().cpu().numpy().tolist()[0]]
            age_pred = age_pred.detach().cpu().item()
            gender_pred = gender[(torch.sigmoid(gender_pred) > 0.5).int().detach().cpu().numpy().tolist()[0][0]]

        result_text = f'{expression_pred} 표정을 한 {int(age_pred)}세 {gender_pred}'
        result_prompt = llama_run(result_text)
        gr.Info('추천 기반 음악 장르와 분위기를 입력해주세요!')
        return result_text, result_prompt

    except Exception as e:
        return f"오류가 발생했습니다: {e}", None
def generate_songs(style, add_prompt,result_output):
    gr.Info('노래 생성 중 입니다!!!')
    file_path=[]
    client=suno.Suno(cookie=cookies)
    songs=client.generate(
        prompt=f'{result_output}, {style}, {add_prompt}, korean',is_custom=False,wait_audio=True
    )
    for song in songs:
        file_path.append(client.download(song=song))
    gr.Info('노래 생성 완료 되었습니다!!!')
    return file_path[0],file_path[1]


#Gradio

In [ ]:

custom_css = """
body {
    font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
    margin: 0;
    padding: 0;
}
h1 {
    color: #1e90ff;
    text-align: center;
}
.label, .output_text, .output_audio {
    font-size: 16px;
    font-weight: bold;
    color: #555;
}

.gr-tabitem {
    background-color: #ffffff;
    border-radius: 5px;
    padding: 20px;
    box-shadow: 0px 0px 10px rgba(0, 0, 0, 0.1);
}
.gr-tabitem.active {
    background-color: #f1f1f1;
}
.center {
    display: flex;
    justify-content: center;
    align-items: center;
    flex-direction: column;
}
.gradio-container {
    background: transparent;
    background: url('https://images.unsplash.com/photo-1445375011782-2384686778a0?q=80&w=1170&auto=format&fit=crop&ixlib=rb-4.0.3&ixid=M3wxMjA3fDB8MHxwaG90by1wYWdlfHx8fGVufDB8fHx8fA%3D%3D') no-repeat center center fixed;
    background-size: cover;
}
"""

description_html = """
<div style="text-align: center; margin: 20px 0;">
    <p style="font-size: 1.2em; color: #333; max-width: 800px; margin: 0 auto;">
        This interface analyzes images uploaded by users to predict gender, age, and emotions. It also creates songs with analysis data.
    </p>
    <p style="font-size: 1.2em; color: #333; max-width: 800px; margin: 10px auto;">
        Click the <strong>Upload Images</strong> button below to upload an image
    </p>
    <p style="font-size: 1.2em; color: #ff69b4; max-width: 800px; margin: 20px auto;">
        Get ready to embark on a musical journey with your face!
    </p>
</div>
"""

js = """
window.onload = function() {
    var h1 = document.createElement('h1');
    h1.id = 'title-text';
    h1.style.fontFamily = "'Dancing Script', cursive";
    h1.style.fontSize = '3em';
    h1.style.textShadow = '2px 2px #ff1493';
    h1.innerText = '\\uD83C\\uDFB5 Music Face \\uD83C\\uDFB5';

    var descriptionContainer = document.querySelector('.gradio-container > div');
    descriptionContainer.insertBefore(h1, descriptionContainer.firstChild);

    var text = h1.innerText;
    h1.innerText = '';

    for (var i = 0; i < text.length; i++) {
        (function(i){
            setTimeout(function(){
                var letter = document.createElement('span');
                letter.style.opacity = '0';
                letter.style.transition = 'opacity 0.5s';
                letter.innerText = text[i];

                h1.appendChild(letter);

                setTimeout(function() {
                    letter.style.opacity = '1';
                }, 50);
            }, i * 250);
        })(i);
    }
}
"""
def reset_interface():
    return None, None, None, None, None

with gr.Blocks(css=custom_css,js=js) as iface:
    gr.Markdown(description_html)
    with gr.Row():
        img_input = gr.Image(type="pil", label="Upload Images", height=400, width=400)
    with gr.Tabs():
        with gr.TabItem("Analysis"):
            result_output = gr.Textbox(label='분석 결과')
            result_prompt_output = gr.Textbox(label='Llama3 기반 추천 결과')
            style_box = gr.Textbox(label="음악 장르", placeholder="Kpop")
            add_box = gr.Textbox(label='추가 프롬프트', placeholder="Female vocals")
        with gr.TabItem("Music1"):
            music1_output = gr.Audio(label='Music1')
        with gr.TabItem("Music2"):
            music2_output = gr.Audio(label='Music2')
    img_input.change(fn=model_run, inputs=img_input, outputs=[result_output, result_prompt_output])
    generate_button = gr.Button('음악 생성')
    clear = gr.Button('다른 이미지 시도')
    generate_button.click(fn=generate_songs, inputs=[style_box, add_box, result_output], outputs=[music1_output, music2_output])
    clear.click(fn=reset_interface, inputs=[], outputs=[img_input, result_output, result_prompt_output, music1_output, music2_output], queue=False)
iface.launch(show_error=True)


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://ce9c539ab752525cf2.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
